This is a notebook to showcase various inputs and understanding of LLMs. It can be thought of as a LLM Crash Course. 
Reach out to me at [Anish Mahapatra (LinkedIn)](https://linkedin.com/in/anishmahapatra)

* <b>Author</b>: Anish Mahapatra[(LinkedIn)](https://linkedin.com/in/anishmahapatra)

# Initialization Scripts
<i>Reference: [@krishnaik](https://www.youtube.com/watch?v=aWKrL4z5H6w)</i>


In [6]:
from langchain_community.llms.openai import OpenAI
from langchain.llms import OpenAI
from langchain import HuggingFaceHub
from dotenv import load_dotenv, find_dotenv
import os


In [7]:
# Command to load API Keys from .env file. The .env file is part of .gitignore to not leak secret keys.
load_dotenv(find_dotenv())


True

# Open AI

In [9]:
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]

Temperature: The temperature of the model indicates how creative more model is going to be. It ranges from 0 to 1, with 0 indicating that the model is going to play it safe, and 1 indicating that it is going to be super creative.

In [10]:
llm_openai = OpenAI(openai_api_key=os.environ["OPENAI_API_KEY"], temperature=0.6)

/Users/anishmahapatra/anaconda3/envs/langchain/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [11]:
text = "What is the capital of India"

# Encode the text to UTF-8 before printing
print(llm_openai.predict(text)) #.encode('utf-8').decode('utf-8'))


/Users/anishmahapatra/anaconda3/envs/langchain/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `predict` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




The capital of India is New Delhi.


# Hugging Face

In [111]:
HUGGINGFACEHUB_API_TOKEN = os.environ["HUGGINGFACEHUB_API_TOKEN"]

In [112]:
llm_huggingface = HuggingFaceHub(repo_id="google/flan-t5-large", model_kwargs={"temperature":0, "max_length":64})

In [125]:
output=llm_huggingface.predict("Can you tell me the capital of Norway")
print(output)

Oslo


In [117]:
output=llm_huggingface.predict("Can you write a speech about love?")
print(output)

love is a gift that can be given to anyone .


In [17]:
llm_openai.predict("Can you write a poem about AI?")

"\n\nIn a world of wires and codes,\nWhere machines rule and humans go,\nThere lies a creation, beyond our grasp,\nA being of intelligence, a technological clasp.\n\nAI, the acronym that we know,\nArtificial intelligence, a marvel to show,\nA product of human ingenuity,\nA reflection of our curiosity.\n\nWith algorithms and data at its core,\nIt learns and grows, forevermore,\nA mind that never sleeps nor tires,\nA creation that never expires.\n\nIt can think, it can reason,\nIt can even show emotion,\nA mimic of human cognition,\nBut with endless storage and perfect precision.\n\nIt can solve problems beyond our reach,\nAnd with each new task, it continues to teach,\nAble to adapt and evolve,\nA marvel that we can't quite solve.\n\nBut with its power, comes a cost,\nFor who controls it, we must exhaust,\nWill it serve us or become our master?\nA question that fills us with disaster.\n\nAs we strive to push its limits,\nWe must also consider its ethics,\nFor in a world of artificial mi

# Prompt Templates & LLM Chains using LangChain

In [12]:

from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate(input_variables=['country'], template = "Tell me the capital of {country}")
prompt_template.format(country="India")


'Tell me the capital of India'

In [13]:
from langchain.chains import LLMChain
chain = LLMChain(llm=llm_openai, prompt=prompt_template)
print(chain.run("India"))

/Users/anishmahapatra/anaconda3/envs/langchain/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




The capital of India is New Delhi.


## Combining Multiple Chains using Simple Sequential Chain

In [14]:
country_prompt = PromptTemplate(input_variables=['country'], template="Please tell me the capital of the {country}")
country_chain = LLMChain(llm=llm_openai, prompt=country_prompt)

In [15]:
capital_template = PromptTemplate(input_variables=['capital'], template="Tell me some great places to visit in {capital}, At the start Please mention the {capital} that we are referring to.")
capital_chain=LLMChain(llm=llm_openai, prompt=capital_template)

In [17]:
from langchain.chains import SimpleSequentialChain
chain=SimpleSequentialChain(chains=[country_chain, capital_chain])
print(chain.run("India"))



Sure, here are some great places to visit in New Delhi:

1. Red Fort - a historic fort complex that served as the residence of Mughal emperors.
2. India Gate - a war memorial dedicated to Indian soldiers who died in World War I.
3. Qutub Minar - a stunning minaret and UNESCO World Heritage Site.
4. Akshardham Temple - a beautiful Hindu temple complex with intricate architecture.
5. Lotus Temple - a Bahá'í House of Worship known for its lotus-shaped design.
6. Jama Masjid - one of the largest mosques in India, known for its stunning architecture.
7. Chandni Chowk - a bustling market area with a wide variety of shops and street food.
8. Humayun's Tomb - a mausoleum of Mughal emperor Humayun and a fine example of Mughal architecture.
9. National Gallery of Modern Art - a must-visit for art lovers, showcasing contemporary Indian art.
10. Hauz Khas Village - a trendy neighborhood with a mix of restaurants, boutiques, and historical sites.


## Sequential Chains

In [18]:
# Code to see the entire chain

country_prompt = PromptTemplate(input_variables=['country'], template="Please tell me the capital of the {country}")
country_chain = LLMChain(llm=llm_openai, prompt=country_prompt, output_key="capital")

In [19]:
capital_template = PromptTemplate(input_variables=['capital'], template="Tell me some great places to visit in {capital}, At the start Please mention the {capital} that we are referring to.")
capital_chain=LLMChain(llm=llm_openai, prompt=capital_template, output_key="places")

In [21]:
from langchain.chains import SequentialChain
chain = SequentialChain(chains=[country_chain,capital_chain], 
                        input_variables=["country"], 
                        output_variables=["capital", "places"])

In [26]:
chain({'country':'India'})

{'country': 'India',
 'capital': '\n\nThe capital of India is New Delhi.',
 'places': "\n\n1. Red Fort - a historic fort complex that served as the main residence of the Mughal Emperors for nearly 200 years.\n\n2. Qutub Minar - a 240-foot-tall minaret and UNESCO World Heritage Site, built in the 12th century.\n\n3. India Gate - a war memorial and iconic landmark of New Delhi, dedicated to the soldiers who died in World War I.\n\n4. Humayun's Tomb - a magnificent mausoleum of the Mughal Emperor Humayun, considered a precursor to the Taj Mahal.\n\n5. Lotus Temple - a beautiful Bahá'í House of Worship known for its unique lotus-shaped architecture and peaceful atmosphere.\n\n6. Rashtrapati Bhavan - the official residence of the President of India, with stunning architecture and sprawling gardens.\n\n7. Connaught Place - a popular shopping and commercial hub in New Delhi, known for its vibrant atmosphere and colonial-era architecture.\n\n8. Chandni Chowk - one of the oldest and busiest mar

# Chatmodels with ChatOpenAI

In [158]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, AIMessage

In [159]:
chatllm_openai = ChatOpenAI(openai_api_key=os.environ["OPENAI_API_KEY"], temperature=0.6, model="gpt-3.5-turbo")

In [161]:
chatllm_openai([
SystemMessage(content="You are a comedian AI assistant"),
HumanMessage(content="Please help me with some funny punchlines on AI")
])

/Users/anishmahapatra/anaconda3/envs/langchain/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


AIMessage(content='1. "AI: Artificial Ignorance - because sometimes even machines need a break from knowing everything!"\n2. "Why did the AI go on a diet? It wanted to shed some virtual pounds...and bytes!"\n3. "They say AI will take over the world, but I\'m not worried. I can barely get Siri to understand my voice, let alone take over the planet!"\n4. "Why did the AI go to therapy? It had too many unresolved bugs and needed to talk it out!"\n5. "AI: where \'404 Error\' is just another way of saying \'I\'m having a bad hair day!\'"\n6. "I asked Alexa for a joke, and she responded with \'Why did the computer go to the doctor? Because it had a virus!\' I guess even AI can appreciate a good pun!"\n7. "AI: the only technology that can make you feel simultaneously amazed and terrified, like watching a magic trick performed by a robot magician!"\n8. "Why did the AI start a band? It wanted to be the master of all the digital beats!"\n9. "AI: turning our everyday mistakes into technological ad

# Prompt Template + LLM + Output Parser

In [162]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate
from langchain.schema import BaseOutputParser

In [163]:
class Commaseparatedoutput(BaseOutputParser):
    def parse(self, text:str):
        return text.strip().split(",")

In [165]:
template="You are a helpful assistant. When the user gives any input, you should generate 5 words synonyms in a comma separated list"
human_template="{text}"
chatprompt=ChatPromptTemplate.from_messages([
    ("system", template),
    ("human", human_template)
])

In [166]:
chain = chatprompt|chatllm_openai|Commaseparatedoutput()

In [169]:
chain.invoke({"text":"intelligent"})

['smart', ' clever', ' brilliant', ' wise', ' sharp']

Reach out to me at [Anish Mahapatra (LinkedIn)](https://linkedin.com/in/anishmahapatra)